#Using Time Travel

### Common usecase: Restoring data-related objects (tables, schemas, and databases) that might have been accidentally or intentionally deleted.

## Unusual Usecase: Time Series Analysis

In [0]:
%sql
drop table if exists uat.default.customer_signup_details;

In [0]:
%sql
CREATE TABLE customer_signup_details (
  userId INT,
  name STRING,
  email STRING,
  signup_source STRING,
  created_at TIMESTAMP
) USING DELTA;

In [0]:
%sql
-- Insert initial data
INSERT INTO customer_signup_details VALUES
(1, 'Alice', 'alice@example.com', 'web', current_timestamp()),
(2, 'Bob', 'bob@example.com', 'mobile', current_timestamp()),
(3, 'Charlie', 'charlie@example.com', 'web', current_timestamp());

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from customer_signup_details;

userId,name,email,signup_source,created_at
1,Alice,alice@example.com,web,2025-04-19T08:14:12.381Z
2,Bob,bob@example.com,mobile,2025-04-19T08:14:12.381Z
3,Charlie,charlie@example.com,web,2025-04-19T08:14:12.381Z


In [0]:
%sql
-- Save this timestamp for time travel
SELECT current_timestamp() AS snapshot_time;

snapshot_time
2025-04-19T08:15:46.903Z


In [0]:
%sql
-- Add more users after a delay
INSERT INTO customer_signup_details VALUES
(4, 'David', 'david@example.com', 'mobile', current_timestamp()),
(5, 'Eva', 'eva@example.com', 'web', current_timestamp());

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from customer_signup_details;

userId,name,email,signup_source,created_at
1,Alice,alice@example.com,web,2025-04-19T08:14:12.381Z
2,Bob,bob@example.com,mobile,2025-04-19T08:14:12.381Z
3,Charlie,charlie@example.com,web,2025-04-19T08:14:12.381Z
4,David,david@example.com,mobile,2025-04-19T08:16:04.807Z
5,Eva,eva@example.com,web,2025-04-19T08:16:04.807Z


In [0]:
%sql
SELECT * FROM customer_signup_details TIMESTAMP AS OF '2025-04-19T08:15:46.903+00:00'

userId,name,email,signup_source,created_at
1,Alice,alice@example.com,web,2025-04-19T08:14:12.381Z
2,Bob,bob@example.com,mobile,2025-04-19T08:14:12.381Z
3,Charlie,charlie@example.com,web,2025-04-19T08:14:12.381Z


In [0]:
%sql
SELECT 
  COUNT(DISTINCT userId) -
  (
    SELECT COUNT(DISTINCT userId)
    FROM customer_signup_details TIMESTAMP AS OF '2025-04-19T08:15:46.903+00:00'
  ) AS new_customers_last_2_minutes
FROM customer_signup_details;

new_customers_last_2_minutes
2


# Time travel simplifies low-cost comparisons between previous versions of data.

In [0]:
%sql
select * from customer_signup_details
except
select * from customer_signup_details TIMESTAMP AS OF '2025-04-19T08:15:46.903+00:00'

userId,name,email,signup_source,created_at
4,David,david@example.com,mobile,2025-04-19T08:16:04.807Z
5,Eva,eva@example.com,web,2025-04-19T08:16:04.807Z
